# Ski Resort Map
Use Folium, a Python wrapper for the Javascript library called Leaflet, to construct a map of the 8 ski resorts I analyzed with a week-to-week slider that visualizes the `overall_resort_score`.

In [2]:
import pandas as pd
import os
import glob
import fastparquet
import folium
from folium.plugins import TimestampedGeoJson
import calendar
import datetime

## Reading in the Data
I used Azure CLI and scp commands to transfer my "weather_scored" parquet file to my local file system so I can work here.

In [3]:
loc_lookup = []
for file in glob.glob('data/*_new.csv'):
    d = pd.read_csv(file)

    locs = d[['ski_resort', 'us_region', 'latitude', 'longitude', 'original_timezone']].drop_duplicates()
    loc_lookup.append(locs)

loc_lookup = pd.concat(loc_lookup)

In [4]:
loc_lookup

,ski_resort,us_region,latitude,longitude,original_timezone
0,The Summit at Snoqualmie,PNW,47.424,-121.417,America/Los_Angeles
286,Mt. Bachelor,PNW,44.002,-121.678,America/Los_Angeles
572,Boyne Mountain,Midwest,45.159,-84.939,America/Detroit
0,Boyne Highlands,Midwest,45.469,-84.922,America/Detroit
0,Mammoth Mountain,Northern California,37.631,-119.032,America/Los_Angeles
286,Sierra-at-Tahoe,Northern California,38.800,-120.081,America/Los_Angeles
572,Copper Mountain,Rockies,38.487,-105.766,America/Denver
858,Snowbird,Rockies,40.581,-111.656,America/Denver


In [5]:
pf = fastparquet.ParquetFile("data/weather_scored/")
df = pf.to_pandas()

In [6]:
ski_weather_data = df.merge(loc_lookup, how="left", on="ski_resort")

## Creating the Map

In [30]:
m = folium.Map(location=[39.83, -98.58], zoom_start=5, tiles="cartodb positron")

In [8]:
for _, row in loc_lookup.iterrows():
    popup_text = f"Region: {row['us_region']}\nTimezone: {row['original_timezone']}"

    folium.Marker(
        location=[(row['latitude']), row['longitude']],
        tooltip=row['ski_resort'],
        popup=popup_text,
        icon=folium.Icon(icon="person-skiing", prefix="fa", )
    ).add_to(m)

In [9]:
m

Next Steps:
1. get the top 3 ski resorts for each week based on `overall_resort_score`
2. Convert the `week_num`'s to actual 2026 date ranges.
3. Make a dictionary to capture the markers with lat/long of the rankings for each `week_num` so it can link to the slider via `TimeSliderChoropleth`. The top 1 should have a gold-colored icon, 2nd should have a silver-colored icon, 3rd should have a bronze-colored icon, and everyone else should have a light gray to give the impression it's grayed out yet still visible. Also, the tooltip should still say the resort's name. The popup should give info on what it's `overall_resort_score` is along with what weather condition you can expect (`weather_main`). If I can give a title to the graph or slider, I would say it's "Week 2: MM-DD-YYYY to MM-DD-YYYY" and fill in the 2025-26 actual dates.

Style ideas: color code the top 3 with gold, silver, and bronze icons when creating Markers.

In [10]:
ski_weather_data['rank'] = ski_weather_data.groupby('week_num')['overall_resort_score'].rank(ascending=False)

Convert the `week_num`'s to actual 2025-26 season date ranges:

In [11]:
cal = calendar.Calendar()
month_range = [(2025, 12)]
end_month = 4
end_year = 2026
month_range.extend([(end_year, m) for m in range(1,end_month+1)])

In [12]:
dates_2526 = {}
week_num = 1
for yr, mo in month_range:
    print(f"Year: {yr}\t Month:{mo}")
    for i, wk in enumerate(cal.monthdatescalendar(yr, mo)):
        # just want to grab the lists of datetime objects where the first element falls within the given 'mo' AND
        # want to ignore the last couple weeks in April since those dates fall beyond the assumed ski season
        if wk[0].month != mo or (mo == end_month and i > 2):
            continue
    
        print(f"This is Week #{week_num}: {wk}")
        dates_2526[week_num] = wk
        week_num += 1

Year: 2025	 Month:12
This is Week #1: [datetime.date(2025, 12, 1), datetime.date(2025, 12, 2), datetime.date(2025, 12, 3), datetime.date(2025, 12, 4), datetime.date(2025, 12, 5), datetime.date(2025, 12, 6), datetime.date(2025, 12, 7)]
This is Week #2: [datetime.date(2025, 12, 8), datetime.date(2025, 12, 9), datetime.date(2025, 12, 10), datetime.date(2025, 12, 11), datetime.date(2025, 12, 12), datetime.date(2025, 12, 13), datetime.date(2025, 12, 14)]
This is Week #3: [datetime.date(2025, 12, 15), datetime.date(2025, 12, 16), datetime.date(2025, 12, 17), datetime.date(2025, 12, 18), datetime.date(2025, 12, 19), datetime.date(2025, 12, 20), datetime.date(2025, 12, 21)]
This is Week #4: [datetime.date(2025, 12, 22), datetime.date(2025, 12, 23), datetime.date(2025, 12, 24), datetime.date(2025, 12, 25), datetime.date(2025, 12, 26), datetime.date(2025, 12, 27), datetime.date(2025, 12, 28)]
This is Week #5: [datetime.date(2025, 12, 29), datetime.date(2025, 12, 30), datetime.date(2025, 12, 31),

In [13]:
dates_2526[4]

[datetime.date(2025, 12, 22),
 datetime.date(2025, 12, 23),
 datetime.date(2025, 12, 24),
 datetime.date(2025, 12, 25),
 datetime.date(2025, 12, 26),
 datetime.date(2025, 12, 27),
 datetime.date(2025, 12, 28)]

Formatting a GeoJSON dictionary to get converted to JavaScript in `TimestampedGeoJson()`:

In [83]:
geojson_data = {
    'type': 'FeatureCollection'
}
feature_data = []

for _, dp in ski_weather_data.iterrows():
    if dp['weather_main'] == "Clear":
        weather_text = "be"
    else:
        weather_text = "have"

    if str(dp['week_num'])[-1] == 1:
        week_text = "st"
    elif str(dp['week_num'])[-1] == 2:
        week_text = "nd"
    elif str(dp['week_num'])[-1] == 3:
        week_text = "rd"
    else:
        week_text = "th"

    # get the 2025-26 actual dates for the week in question
    date_list = dates_2526[dp['week_num']]
    first_date = date_list[0].strftime("%b %d, %Y")
    last_date = date_list[-1].strftime("%b %d, %Y")
    date_list_iso = [d.strftime("%Y-%m-%d") for d in date_list]

    # figure out the selected_color based on the rank
    if dp['rank'] == 1:
        selected_color = '#EFBF04' # gold
    elif dp['rank'] == 2:
        selected_color = '#A9A9A9' # silver
    elif dp['rank'] == 3:
        selected_color = '#CD7F32' # bronze
    else:
        selected_color = '#FFFFFF' # white

    popup_text = f"<h1>{dp['ski_resort']}</h1><h2>Score: <b>{dp['overall_resort_score']:.2f}</b>/100</h2><br>In the {dp['us_region']} region, you can expect the weather to {weather_text} <b>{dp['weather_main']}</b> during the {dp['week_num']}{week_text} week of ski season, which is <b> {first_date} to {last_date}</b>."


    row = {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': [dp['longitude'], dp['latitude']]
        },
        'properties': {
            'times': date_list_iso,
            'icon': 'circle',
            'style': {
                'color': selected_color,
                'radius': 20
            },
            'tooltip': popup_text
        }
    }

    feature_data.append(row.copy())

# add this list of features to geojson_data dictionary
geojson_data['features'] = feature_data

In [99]:
geojson_data

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [-121.417, 47.424]},
   'properties': {'times': ['2026-01-12',
     '2026-01-13',
     '2026-01-14',
     '2026-01-15',
     '2026-01-16',
     '2026-01-17',
     '2026-01-18'],
    'icon': 'circle',
    'style': {'color': '#A9A9A9', 'radius': 20, 'font-size': '14px'},
    'tooltip': '<h1>The Summit at Snoqualmie</h1><h2>Score: <b>62.57</b>/100</h2><br>In the PNW region, you can expect the weather to have <b>Clouds</b> during the 7th week of ski season, which is <b> Jan 12, 2026 to Jan 18, 2026</b>.'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [-84.922, 45.469]},
   'properties': {'times': ['2026-03-23',
     '2026-03-24',
     '2026-03-25',
     '2026-03-26',
     '2026-03-27',
     '2026-03-28',
     '2026-03-29'],
    'icon': 'circle',
    'style': {'color': '#EFBF04', 'radius': 20, 'font-size': '14px'},
    'tooltip': '<h1>Boyne Highlands</h1>

In [95]:
base_map = folium.Map(location=[39.83, -98.58], zoom_start=5, tiles="cartodb positron")

In [96]:
TimestampedGeoJson(
    geojson_data,
    period="P1D",
    add_last_point=True,
    auto_play=False,
    loop=False,
    max_speed=1,
    loop_button=True,
    date_options="YYYY-MM-DD",
    time_slider_drag_update=True,
    duration="P1D",
).add_to(base_map)

In [97]:
for _, row in loc_lookup.iterrows():
    popup_text = f"Region: {row['us_region']}\nTimezone: {row['original_timezone']}"

    folium.Marker(
        location=[(row['latitude']), row['longitude']],
        tooltip=row['ski_resort'],
        popup=popup_text,
        icon=folium.Icon(icon="person-skiing", prefix="fa", )
    ).add_to(base_map)

In [98]:
base_map

In [89]:
base_map.save('output/ski_map.html')